Table of Contents:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import csv
import json

Input Constants from .csv

NOTE: NEED TO ADD A CONSTANT TO CHOOSE EMRAX OR AMK MOTORS (AND THEN IT ONLY NEEDS TO READ THAT DATA - AND IT WILL ONLY ANALYZE CERTAIN DRIVETRAIN SECTIONS AS APPLICABLE - since the emrax data is less thorough...)

In [2]:
# USER-INPUT CONSTANTS
# TRACK FILE
TRACK = None
regen_on = None                 # True/False - Regen on or off
numLaps = None

# CAR CONSTANTS
mass = None                     # kg - CAR MASS
Af = None                       # m^2 - FRONTAL AREA
mu_rr = None                    # COEFFICIENT OF ROLLING RESISTANCE

# BATTERY CONSTANTS
# DEPENDS ON STARTING CONDITIONS
initial_SoC = None              # % - INITIAL STATE OF CHARGE
starting_voltage = None         # V - INITIAL PACK VOLTAGE
capacity0 = None                # Ah - INITIAL PACK CAPACITY
# DEPENDS ON BATTERY CHOICE
max_capacity = None             # Ah
n_converter = None              # converter efficiency
cell_max_voltage = None         # V - MAX CELL VOLTAGE
cell_min_voltage = None         # V - MIN CELL VOLTAGE
cell_nominal_voltage = None
num_series_cells = None         # NUMBER OF SERIES ELEMENTS
num_parallel_cells = None       # NUMBER OF PARALLEL CELLS
single_cell_ir = None           # Ohms - CELL INTERNAL RESISTANCE
max_CRate = None                # Max C-Rate
cell_mass = None                # Mass of single cell
battery_cv = None               # Specific heat capacity of battery
# !!!
cell_water_area = None          # m^2 - WATER COOLING SURFACE OF CELL
cell_aux_factor = None          # kg/kWh - SEGMENT AUXILLARY MASS/ENERGY

# MOTOR CONSTANTS
max_motor_torque = None         # Nm - MAX MOTOR TORQUE
max_motor_rpm = None            # rpm - MAX MOTOR RPM
max_power = None                # W - MAX POWER - AS DEFINED BY USER (FOR POWER LIMITING)

# TRACTION CONSTANTS
max_speed_kmh = None            # km/h - MAX SPEED
traction_speed = None           # km/h - MAX SPEED AROUND RADIUS IN TRACTION TEST
traction_radius = None          # m - RADIUS OF TRACTION TEST
longitudinal_friction = None    # Coefficient of longitudinal friction
brake_decel = None              # m/s^2 - BRAKING DECELERATION RATE

# !!! 
# THERMAL CONSTANTS
heatsink_air_area = None        # m^2 - AIR COOLING SURFACE OF CELL
heatsink_mass = None            # kg - TOTAL PACK HEATSINK MASS
heatsink_cv = None              # J/C*kg - HEATSINK MATERIAL SPECIFIC HEAT
air_temp = None                 # C - CONSTANT ASSUMED AIR TEMP
water_temp = None               # C - CONSTANT ASSUMED WATER TEMP
air_htc = None                  # W/C*m^2 - ASSUMED CONSTANT AIR HTC
water_htc = None                # W/C*m^2 - ASSUMED CONSTANT WATER HTC
thermal_resistance_SE = None    # K/W - ASSUMED SERIES ELEMENT THERMAL RESISTANCE
air_factor_m = None             # kg/kg AIR COOLING MASS PER BATTERY MASS
water_factor_m = None           # kg/kg WATER COOLING MASS PER BATTERY MASS

######################################################################
# This allows an external user to have control over the constants without touching the code

# Open .csv and take constants as input:
filename = "LapSimConstants.csv"

# Open the file
with open(filename, 'r', newline='') as infile:
    reader = csv.reader(infile)
    dataList = list(reader)
    dataList.pop(0)             # Remove title row

    # convert to array
    dataArray = np.array(dataList)

    # Take out the valuable columns and convert to floats as necessary
    value_name = dataArray[2:,0]
    value = dataArray[2:,2]
    value = np.asarray(value, dtype = float)

    track_name = dataArray[0,0]
    track = dataArray[0,2]

    regen_name = dataArray[1,0]
    regen = dataArray[1,2]

# Now create variables for everything
for x, y in zip(value_name, value):
    globals()[x] = y

globals()[track_name] = track
globals()[regen_name] = regen

Calculated Constants

In [3]:
# CONSTANTS - SHOULD NOT NEED CHANGING
delta_d = 0.01           # distance interval - m
g = 9.81                # m/s^2
GR = 4.2                # Gear Ratio
wheel_diameter = 18 * 0.0254    # m
wheel_radius = wheel_diameter / 2
rho_air = 1.23         # air density: kg / m^3
v_air = 0               # air velocity: m/s
radsToRpm = 1 / (2 * math.pi) * 60    # rad/s --> rpm
Cd = 1                  # drag coefficient

# Battery Pack - Calculated Values
num_cells = num_series_cells * num_parallel_cells
pack_nominal_voltage = cell_nominal_voltage * num_series_cells # V
total_pack_ir = single_cell_ir / num_parallel_cells * num_series_cells  # ohms
# !!! Total known energy is approximately SoC * nominal voltage * max capacity
knownTotalEnergy = initial_SoC * capacity0 * pack_nominal_voltage / 1000  # kWh
pack_min_voltage = cell_min_voltage * num_series_cells  # V

# !!!
# Car Mass - Calculated Values
total_cell_mass = cell_mass*num_cells # kg
cooled_cell_mass = total_cell_mass*(1 + air_factor_m + water_factor_m) # kg
cell_aux_mass = cell_aux_factor*(capacity0 * pack_nominal_voltage / 1000) # kg
mass = mass + cooled_cell_mass + cell_aux_mass + heatsink_mass # kg

# !!! 
# Thermals - Calculated Values
battery_heat_capacity = battery_cv*cell_mass # J/C
air_tc = air_htc*heatsink_air_area  # W/C
water_tc = water_htc*cell_water_area # W/C
air_thermal_resistance = 1 / air_tc  # K/W
heatsink_temp_0 = air_temp           # C
batteryTemp0 = air_temp             # C - starting temperature of battery pack (may change if necessary)

# Traction Constants
# at 30 km/h, we travelled around a 5 m radius circle
a_centrip = (traction_speed * 1000 / 3600)**2 / traction_radius      # v^2 / r (convert to m/s)
test_mass = 225                             # kg - car mass used in testing
F_friction = test_mass * a_centrip          # calculate the friction force
mu_f = F_friction / (test_mass * g)         # calculate the friction coefficient
max_speed = max_speed_kmh / 3.6             # m/s
max_traction_force = mass * g * longitudinal_friction   # N
F_friction = mu_f * mass * g                # friction force based on the evaluated car mass.    

Import Datasets
1) AMK Motor
2) Emrax Motor
3) Track Data

In [4]:
############
# AMK Motor
# Import AMK Data
in_json = 'AMK_data.json'
in_json = open(in_json)
in_json = in_json.read()
AMK_dict = json.loads(in_json)

# Converting list/dict storage into Dataframe and Array Storage
AMK_dict['ShaftTorque'] = pd.DataFrame(AMK_dict['ShaftTorque'])
AMK_dict['PowerFactor'] = pd.DataFrame(AMK_dict['PowerFactor'])
AMK_dict['TotalLosses'] = pd.DataFrame(AMK_dict['TotalLosses'])
AMK_dict['PeakTorqueSpeed'] = np.array(AMK_dict['PeakTorqueSpeed'])
AMK_dict['ContTorqueSpeed'] = np.array(AMK_dict['ContTorqueSpeed'])

############
# Emrax Motor
# Convert .json to dictionary
in_json = 'emrax_data.json'
in_json = open(in_json)
in_json = in_json.read()
emrax_dict = json.loads(in_json)

# Converting list/dict storage into Dataframe and Array storage
emrax_dict['Motor Efficiency'] = pd.DataFrame(emrax_dict['Motor Efficiency'])
emrax_dict['PeakTorqueSpeed'] = np.array(emrax_dict['PeakTorqueSpeed'])
emrax_dict['ContTorqueSpeed'] = np.array(emrax_dict['ContTorqueSpeed'])

# Additional emrax data - also need a function handle for the Torque/Current Characteristic
phases = 3
pole_pairs = 10
pm = emrax_dict['lambda_pm']

# Function handle for managing torque --> Current changes (if necessary)
torque_current = lambda I: phases / 2 * pole_pairs * pm * I

############
# Track Data
infile = "Sim_Autocross.csv"
trackData = pd.read_csv(infile)

Pre-iteration analysis:
1) Track maximum velocities
2) Braking iteration

In [8]:
###################
# Track maximum velocities

b_vec = (trackData['Radius'] == 0) * 100000       # Create boolean vector for input data - and update the radius to be VERY large on straights
trackData['Radius'] = trackData.loc[:,'Radius'] + b_vec   # Then we update the radius vector

# And now... add a column to the dataframe with the maximum speed in each section (this is kinda like doing vector operations in MATLAB)
trackData = trackData.assign(MaxVelocity = np.sqrt(a_centrip * trackData['Radius']))

###################
# Braking Iteration

# Initialize a dictionary to store all of the braking informtion
TrackLength = int(trackData.shape[0]) # Provides the length of a column of the df

sectors = list(np.linspace(0, TrackLength, num = TrackLength, endpoint=False)) # keys for the number of sectors

brakeDict = dict.fromkeys(sectors, None)

# Dataframe parameters:
dictList = ['Time','Distance','Velocity']

# Iterate through the track loop to determine the braking data at each point
for i in range(0, TrackLength):
    # Find sector length
    sectorLength = trackData.loc[trackData.index[i], 'Section Length']

    # Find sector vector length
    vectorLength = int(sectorLength / delta_d)

    # Initialize brakeSector_dict
    brakeSector_dict = dict.fromkeys(dictList)
    
    # add empty zero vectors...
    for k in range(0, len(dictList)):
        brakeSector_dict[dictList[k]] = np.zeros(vectorLength)

    # Initialize values in dict
    brakeSector_dict['Time'][vectorLength - 1] = 0
    brakeSector_dict['Distance'][vectorLength - 1] = sectorLength

    # Initialize the Max Velocity (Taking into account edge cases)
    if i != (TrackLength - 1):
        brakeSector_dict['Velocity'][vectorLength - 1] = trackData.loc[trackData.index[i+1], 'MaxVelocity']
    else:
        brakeSector_dict['Velocity'][vectorLength - 1] = trackData.loc[trackData.index[0], 'MaxVelocity']

    # Fill up the distance, time, and velocity from the back of the dataframe
    for j in reversed(range(1, vectorLength)):
        # Iteration for distance, time, and velocity
        # First: solve for delta time:
        poly_coeffs = np.array([1/2, brakeSector_dict['Velocity'][j], -delta_d])          # coefficients: p[0] * x^n + ... + p[n]
        brakeSector_dict['Time'][j - 1] = np.roots(poly_coeffs)[1]                            # This is the delta_t

        # then for v1
        brakeSector_dict['Velocity'][j - 1] = brakeSector_dict['Velocity'][j] + brake_decel * brakeSector_dict['Time'][j]

        # then for distance
        brakeSector_dict['Distance'][j - 1] = brakeSector_dict['Distance'][j] + delta_d

    # FINALLY: save the dataframe into the brakeDict
    brakeDict[sectors[i]] = brakeSector_dict

# Now plot the data to check the reality of it...
print('debug pause')

debug pause


At each sector, we want to do this:
* Determine the sector length
* Based on the delta_distance interval, determine the number of entries required
* Pre-allocate the data frame of the determine length with "None"
* The dataframe should have columns: distance, time, and velocity

* No - just fill up all the distance, time, and velocity arrays backwards - that'll be the easiest way to go about this.
